# Data Mining -  Computer Assignment 1
Mohammad Saadati - 
_810198410_

## Introduction
The goal of this computer assignment is review the data set of COVID-19, pre-processing, illustration and express our analysis based on the illustrations performed. This data set have information about patients, new tests, patient deaths and some features related to countries are given separately by day.

## Import Libraries
First of all, we import necessary libraries to use their functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from scipy.stats import norm
from shapely.geometry import LineString
import jdatetime
import datetime

## Preprocessing
First we load csv file as a DataFrame using pandas library.

In [71]:
df = pd.read_csv("CA1_Dataset.csv")
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165631,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,368.429,5393.0,1.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165632,ZWE,Africa,Zimbabwe,2022-02-27,235803.0,0.0,350.143,5393.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165633,ZWE,Africa,Zimbabwe,2022-02-28,236380.0,577.0,401.286,5395.0,2.0,1.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
165634,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


### Question 1
In this part first we use two Pandas function (isna and sum) to count number of missing values at each column.

In [72]:
df.info()
# pd.set_option('display.max_rows',None)
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165636 entries, 0 to 165635
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    165636 non-null  object 
 1   continent                                   155719 non-null  object 
 2   location                                    165636 non-null  object 
 3   date                                        165636 non-null  object 
 4   total_cases                                 162606 non-null  float64
 5   new_cases                                   162464 non-null  float64
 6   new_cases_smoothed                          160480 non-null  float64
 7   total_deaths                                144793 non-null  float64
 8   new_deaths                                  144833 non-null  float64
 9   new_deaths_smoothed                         142734 non-null  float64
 

iso_code                                        0
continent                                    9917
location                                        0
date                                            0
total_cases                                  3030
                                            ...  
human_development_index                     29953
excess_mortality_cumulative_absolute       159940
excess_mortality_cumulative                159940
excess_mortality                           159940
excess_mortality_cumulative_per_million    159940
Length: 67, dtype: int64

Now use fillna function to fill out the missing values in the given column.

In [73]:
unique_continent = df['continent'].unique()

for column in df:
    if df[column].isna().sum() > 0:
        if df[column].dtypes != object:
            df[column].fillna(df[column].mode()[0], inplace=True)
        elif column == "continent":
            df['continent'] = df.apply(lambda row: row['location'] if (pd.isna(row['continent']) and (row['location'] in unique_continent)) 
                                       else ('Undefined' if pd.isna(row['continent']) else row['continent']) , axis=1)
        elif column == "tests_units":
            df['tests_units'] = df.apply( lambda row : df['tests_units'].mode()[0] if (df[df['location'] == row['location']]['tests_units'].isnull().values.all())
                                        else (df[df['location'] == row['location']]['tests_units'].mode()[0] if pd.isna(row['tests_units'])
                                        else row['tests_units']), axis = 1)

df.to_csv("df_p1_q1.csv")
df.isna().sum()

KeyboardInterrupt: 

#### Advantages and disadvantages of using mean for missing values:
Missing values in your data do not reduce your sample size, as it would be the case with listwise deletion (the default of many statistical software packages, e.g. R, Stata, SAS or SPSS). Since mean imputation replaces all missing values, you can keep your whole database.

Replacing missing data by the mean of nonmissing data causes the population SD to be underestimated, but may also obscure important features of the population from which the data were sampled. Another possible disadvantage with using the mean for missing values is that the reason the values are missing in the first place could be dependent on the missing values themselves. (This is called missing not at random.)

Advantages : Easy to apply - Mean will not change

Disadvantages : Results may not be accurate - For large amount of NaN data this method can increase mod value significantly and cause error in results - The variance will decrease

### Question 2

In [68]:
pd.reset_option('display.max_rows')

### Question 3
In this part we use **jdatetime.date.fromgregorian** function from `jdatetime` library to convert gregorian date to shamsi date and store the result in *shamsi_date* column of a new dataframe called **df_p1_q3**.

In [8]:
def convert_gregorian_date_to_shamsi_date(gregorian_date):
    y, m , d = gregorian_date.split("-")
    return str(jdatetime.date.fromgregorian(day=int(d),month=int(m),year=int(y)))

df_p1_q3 = df.copy()
df_p1_q3['shamsi_date'] = df_p1_q3['date']

for index, row in df_p1_q3.iterrows():
    shamsi_date = convert_gregorian_date_to_shamsi_date(row['shamsi_date'])
    df_p1_q3.at[index,'shamsi_date'] = shamsi_date

df_p1_q3.to_csv("df_p1_q3.csv")
df_p1_q3

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,shamsi_date
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-05
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-06
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-07
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-08
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,11492.440366,57400.29315,171.252111,172.811556,...,32.778255,37.746,0.5,64.83,0.511,37613.01782,9.404336,15.967077,972.197816,1398-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165631,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,368.429000,5393.00000,1.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-07
165632,ZWE,Africa,Zimbabwe,2022-02-27,235803.0,0.0,350.143000,5393.00000,0.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-08
165633,ZWE,Africa,Zimbabwe,2022-02-28,236380.0,577.0,401.286000,5395.00000,2.000000,1.286000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-09
165634,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000000,5395.00000,0.000000,1.000000,...,30.700000,36.791,1.7,61.49,0.571,37613.01782,9.404336,15.967077,972.197816,1400-12-10


### Question 4
Redundant attributes may be able to be detected by correlation analysis and covariance analysis

### Question 5

### Question 6

### Question 7

## Display data

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

### Question 6

### Question 7

### Question 8

## Bonus questions

### Question 1

### Question 2